<a href="https://colab.research.google.com/github/adrj67/Aprendizaje_NO_supervisado_Clustering/blob/miprim/Mi5_1_Aprendizaje_no_supervisado_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Librerias

In [ ]:
!pip install -q kaggle

In [ ]:
!pip install pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 50.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681833 sha256=9f4b7bc9370c2ac4e0ba666504166b2125de58cee70e9e6dfa6f578ffddc7e32
  Stored in directory: /root/.cache/pip/wheels/03/57/cc/290c5252ec97a6d78d36479a3c5e5ecc76318afcb241ad9dbe
Successfully built pycountry


In [ ]:
# Display
from IPython.display import HTML, display

# Manipulacion de datos, arreglos, etc
import pandas as pd
import numpy as np

# Visualizacion
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pycountry

# Framework
from sklearn.datasets import make_blobs # Crear datasets sinteticos
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler # Para escalar los datos
from sklearn.decomposition import PCA

##Data set real

Antes de implementar cualquier modelo es necesario preparar los datos para poder alimentar el modelo de manera correcta, esto conlleva:

* Analizar nuestro dataset
* Identificar datos faltantes
* Identificar outliers
* Tratar los datos faltantes, outliers, etc
* Escalar los datos
* Reducir la dimensionalidad de ser posible

A modo de guia general podemos considerar los puntos anteriores

---
#### El dataset que usaremos sera de analisis que se hicieron en los paises, implementar KMeans servira para ver como pueden agruparse los paises segun las metricas que contienen.
---

In [ ]:
from google.colab import files

1. ir a Kaggle>account y con el mouse bajamos hasta la opcion API.
2. Una vez alli, hacemos click en la opcion Create New Appi Token, lo cual descargara un archivo .json que nos permitira conectarnos a la API de Kaggle.
3. Ahora usaremos el modulo de files para cargar el archivo en nuestro sistema local.

In [ ]:
# Sube un archivo desde mi PC
files.upload()

Saving Country-data.csv to Country-data.csv


{'Country-data.csv': b'country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp\r\nAfghanistan,90.2,10,7.58,44.9,1610,9.44,56.2,5.82,553\r\nAlbania,16.6,28,6.55,48.6,9930,4.49,76.3,1.65,4090\r\nAlgeria,27.3,38.4,4.17,31.4,12900,16.1,76.5,2.89,4460\r\nAngola,119,62.3,2.85,42.9,5900,22.4,60.1,6.16,3530\r\nAntigua and Barbuda,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200\r\nArgentina,14.5,18.9,8.1,16,18700,20.9,75.8,2.37,10300\r\nArmenia,18.1,20.8,4.4,45.3,6700,7.77,73.3,1.69,3220\r\nAustralia,4.8,19.8,8.73,20.9,41400,1.16,82,1.93,51900\r\nAustria,4.3,51.3,11,47.8,43200,0.873,80.5,1.44,46900\r\nAzerbaijan,39.2,54.3,5.88,20.7,16000,13.8,69.1,1.92,5840\r\nBahamas,13.8,35,7.89,43.7,22900,-0.393,73.8,1.86,28000\r\nBahrain,8.6,69.5,4.97,50.9,41100,7.44,76,2.16,20700\r\nBangladesh,49.4,16,3.52,21.8,2440,7.14,70.4,2.33,758\r\nBarbados,14.2,39.5,7.97,48.7,15300,0.321,76.7,1.78,16000\r\nBelarus,5.5,51.4,5.61,64.5,16200,15.1,70.4,1.49,6030\r\nBelgium,4.5,76.4,10.7,74.7,

In [ ]:
# Creamos una carpeta oculta en el sistema de archivos
! mkdir ~/.kaggle

# Copiamos el archivo con las credenciales
! cp kaggle.json ~/.kaggle/

# Cambiamos permisos para poder leer las credenciales
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Prueba para la conexioncon la api de kaggle
! kaggle datasets list

---
#### Para descargar el dataset que nosotros querramos simplemente, vamos a donde esta y ponemos "copy api command"
---

In [ ]:
! kaggle datasets download -d rohan0301/unsupervised-learning-on-country-data.zip to /content

In [ ]:
# Conectamos Google Drive para poder ver el archivo
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!mkdir '/content/drive/My Drive/Colab Notebooks/db/'

In [ ]:
# Descomprimimos el archivo en la carpeta creada
!unzip unsupervised-learning-on-country-data.zip -d '/content/drive/My Drive/Colab Notebooks/db/'

unzip:  cannot find or open unsupervised-learning-on-country-data.zip, unsupervised-learning-on-country-data.zip.zip or unsupervised-learning-on-country-data.zip.ZIP.


---
### Analisis
---

Pais: Nombre del pais //
child_mort: Muerte de niños menores de cinco años por cada 1000 nacidos vivos //
exportaciones: Exportaciones de bienes y servicio per capita, como porcentaje del PBI per capita //
Salud: gasto total de pais per capita, como porcentaje del PBI per capita //
importaciones: Importacion de bienes y servicios per capita, como porcentaje del PBI per capita //
Ingresos: ingresos netos por personas //
Inflacion: medida de la tasa de crecimiento anual del total de PBI //
life_expec: Espectativa de vida //
total_fer: cantidad de niños que naceran de cada mujer segun la tasa de fertilidad por edad //
PBIpp: PBI per capita

In [ ]:
# Cargamos el dataset
df_country_raw = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/db/Country-data.csv')

In [ ]:
# Vemos algunas filas...
df_country_raw.head(10)

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,Afghanistan,90.2,10.0,7.58,44.9,1610,9.440,56.2,5.82,553
1,Albania,16.6,28.0,6.55,48.6,9930,4.490,76.3,1.65,4090
2,Algeria,27.3,38.4,4.17,31.4,12900,16.100,76.5,2.89,4460
3,Angola,119.0,62.3,2.85,42.9,5900,22.400,60.1,6.16,3530
4,Antigua and Barbuda,10.3,45.5,6.03,58.9,19100,1.440,76.8,2.13,12200
5,Argentina,14.5,18.9,8.10,16.0,18700,20.900,75.8,2.37,10300
6,Armenia,18.1,20.8,4.40,45.3,6700,7.770,73.3,1.69,3220
7,Australia,4.8,19.8,8.73,20.9,41400,1.160,82.0,1.93,51900
8,Austria,4.3,51.3,11.00,47.8,43200,0.873,80.5,1.44,46900
9,Azerbaijan,39.2,54.3,5.88,20.7,16000,13.800,69.1,1.92,5840


In [ ]:
df_country_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     167 non-null    object 
 1   child_mort  167 non-null    float64
 2   exports     167 non-null    float64
 3   health      167 non-null    float64
 4   imports     167 non-null    float64
 5   income      167 non-null    int64  
 6   inflation   167 non-null    float64
 7   life_expec  167 non-null    float64
 8   total_fer   167 non-null    float64
 9   gdpp        167 non-null    int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 13.2+ KB


In [ ]:
# Cambiamos el tipo de dato de 'objeto' a 'categorico'
df_country_raw["country"] = df_country_raw["country"].astype("category")

In [ ]:
# Trae solo los datos numericos
df_country_raw.describe()

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
count,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000
mean,38.270060,41.108976,6.815689,46.890215,17144.688623,7.781832,70.555689,2.947964,12964.155689
std,40.328931,27.412010,2.746837,24.209589,19278.067698,10.570704,8.893172,1.513848,18328.704809
min,2.600000,0.109000,1.810000,0.065900,609.000000,-4.210000,32.100000,1.150000,231.000000
25%,8.250000,23.800000,4.920000,30.200000,3355.000000,1.810000,65.300000,1.795000,1330.000000
50%,19.300000,35.000000,6.320000,43.300000,9960.000000,5.390000,73.100000,2.410000,4660.000000
75%,62.100000,51.350000,8.600000,58.750000,22800.000000,10.750000,76.800000,3.880000,14050.000000
max,208.000000,200.000000,17.900000,174.000000,125000.000000,104.000000,82.800000,7.490000,105000.000000


In [ ]:
# Para ver si hay datos duplicados
df_country_raw[df_country_raw.duplicated()]

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp


In [ ]:
# para mostrar solo los 'categoricos'
df_country_raw.describe(include="category")

,country
count,167
unique,167
top,Afghanistan
freq,1


In [ ]:
display(df_country_raw['inflation'].max()) #obtenemos el valor maximo en inflacion
display(df_country_raw['inflation'].idxmax()) #obtenemos el id donde se encuentra el valor maximo en inflacion
pd.DataFrame(df_country_raw.iloc[df_country_raw["inflation"].idxmax()]).transpose() # iloc ubica por indice

104.0

113

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
113,Nigeria,130.0,25.3,5.07,17.4,5150,104.0,60.5,5.84,2330


In [ ]:
df_country_raw["child_mort"].mean() # media o promedio (suma de todos los valores dividido la cantidad de datos)

38.27005988023952

In [ ]:
df_country_raw["child_mort"].median() # mediana (puestos todos en fila este numero estaria a la mitad)

19.3

In [ ]:
# @title Histogramas
# minuto 8:58
fig = go.Figure().set_subplots(3, 3, horizontal_spacing=0.1)
i = 1
j = 1

for idx, name in enumerate(df_country_raw.describe().columns):
  fig.add_trace(go.Histogram(x=df_country_raw[name]), row=i, col=j)
  #Update xaxis properties
  fig.update_xaxes(title_text=f"{name}", row=i, col=j)
  fig.add_vline(
      x=df_country_raw.describe()[f"{name}"][1].round(2),
      line_color ="red",
      line_dash = "dash",
      row=i,
      col=j).add_vline(
      x=df_country_raw[f"{name}"].median().round(2),
      line_color = "green",
      line_dash = "dash",
      row=i,
      col=j)

  if j==3:
    i+=1
    j=1
  else:
    j+=1

fig.update_layout(title='Histograma con mediana (verde) y media (roja)', showlegend=False, height=700, width=1100)
fig.show()

* child_mort: tiene una distribucion sesgada a la derecha.
* exportaciones: tiene una distribucion sesgada a la derecha.
* salud: tiene una distribucion sesgada a la derecha.
* import: tiene una distribucion sesgada a la derecha.
* Ingresos: tiene una distribucion sesgada a la derecha.
* inflacion: tiene una distribucion sesgada a la derecha.
* life_expect: tiene una distribucion sesgada a la izquierda.
* total_fert (fertilidad): tiene una distribucion sesgada a la derecha.
* gdpp (pbi per capita): tiene una distribucion sesgada a la derecha.

In [ ]:
# 11:24 - grafico para ver los outlier - Box plot
f = go.Figure()
for i, name in enumerate(df_country_raw.columns):
  f.add_trace(go.Box(y=df_country_raw[f"{name}"], name=f"{name}", opacity=0.5))
f.update_layout(height=900, width=1000, title_text="Box Plots", paper_bgcolor="#e4e9f7")

# Seleccionando 'inflacion' y 'head' podemos ver los datos atipicos de inflacion que estan por fuera del bloque

In [ ]:

#======================================================================
# Funcion para graficar la matriz de correlacion con plotly
#======================================================================
def corr_matrix(df):
  # Calcular la matriz de correlación
  correlation_matrix = df.corr().round(2)

  # Crear una visualización interactiva usando Plotly
  fig = px.imshow(correlation_matrix,
                x=correlation_matrix.columns,
                y=correlation_matrix.columns,
                color_continuous_scale='Viridis',
                text_auto=True)
  fig.update_layout(title='Matriz de correlacion', showlegend=False, height=800, width=800)
  fig.show()


In [ ]:
#  matriz de correlacion con plotly
corr_matrix(df_country_raw.drop(['country'], axis=1))

# cuanto mas cerca de 1, mayor correlacion (0.9, 0.8...)
# Su valor está comprendido en el rango [+1 , -1]. Siendo +1 una correlación positiva perfecta y -1 una correlación negativa perfecta.

---
## Escalado
---

Esto es importante ya que el algoritmo que vamos a utilizar trabaja con distancias.
El escalado nos permite mantener todos los datos dentro de un mismo rango evitando asi el sesgo.

In [ ]:
# Minuto 18:29
# Creamos el objeto para escalar
scaler = StandardScaler()


In [ ]:
df_country_scaled = scaler.fit_transform(df_country_raw.drop("country", axis=1))
df_country_scaled = pd.DataFrame(df_country_scaled, columns=df_country_raw.drop("country", axis=1).columns)
df_country_scaled

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,1.291532,-1.138280,0.279088,-0.082455,-0.808245,0.157336,-1.619092,1.902882,-0.679180
1,-0.538949,-0.479658,-0.097016,0.070837,-0.375369,-0.312347,0.647866,-0.859973,-0.485623
2,-0.272833,-0.099122,-0.966073,-0.641762,-0.220844,0.789274,0.670423,-0.038404,-0.465376
3,2.007808,0.775381,-1.448071,-0.165315,-0.585043,1.387054,-1.179234,2.128151,-0.516268
4,-0.695634,0.160668,-0.286894,0.497568,0.101732,-0.601749,0.704258,-0.541946,-0.041817
...,...,...,...,...,...,...,...,...,...
162,-0.225578,0.200917,-0.571711,0.240700,-0.738527,-0.489784,-0.852161,0.365754,-0.546913
163,-0.526514,-0.461363,-0.695862,-1.213499,-0.033542,3.616865,0.546361,-0.316678,0.029323
164,-0.372315,1.130305,0.008877,1.380030,-0.658404,0.409732,0.286958,-0.661206,-0.637754
165,0.448417,-0.406478,-0.597272,-0.517472,-0.658924,1.500916,-0.344633,1.140944,-0.637754


---

## <h2 align="center"> PCA </h2>
<h4 align="center"> Analisis de componentes principales </h4>
<h5 align="center"> Reducccion de dimensionalidad </h5>

---

Esto es ajeno al cluster, pero nos ayuda con el problema que tenemos al tener muchas dimensiones, el algoritmo basicamente consiste en tomar todas las dimensiones que posee el dataset y reducirlas sin perder mucha varianza (informacion).

In [ ]:
pca = PCA() # Creamos el objeto PCA
pca.fit(df_country_scaled)

PCA()

In [ ]:
pca.explained_variance_ratio_ * 100

array([45.95173979, 17.18162565, 13.0042589 , 11.05316181,  7.34021142,
        2.48423468,  1.2604304 ,  0.98128174,  0.74305562])

In [ ]:
pca.components_

array([[-0.41951945,  0.28389698,  0.15083782,  0.16148244,  0.39844111,
        -0.19317293,  0.42583938, -0.40372896,  0.39264482],
       [ 0.19288394,  0.61316349, -0.24308678,  0.67182064,  0.02253553,
        -0.00840447, -0.22270674,  0.15523311, -0.0460224 ],
       [-0.02954353,  0.14476069, -0.59663237, -0.29992674,  0.3015475 ,
         0.64251951,  0.11391854,  0.01954925,  0.12297749],
       [ 0.37065326,  0.00309102,  0.4618975 , -0.07190746,  0.39215904,
         0.15044176, -0.20379723,  0.37830365,  0.53199457],
       [-0.16896968,  0.05761584,  0.51800037,  0.25537642, -0.2471496 ,
         0.7148691 ,  0.1082198 , -0.13526221, -0.18016662],
       [ 0.20062815, -0.05933283,  0.00727646, -0.03003154,  0.16034699,
         0.06628537, -0.60112652, -0.75068875,  0.01677876],
       [-0.07948854, -0.70730269, -0.24983051,  0.59218953,  0.09556237,
         0.10463252,  0.01848639,  0.02882643,  0.24299776],
       [-0.68274306, -0.01419742,  0.07249683, -0.02894642,  0

In [ ]:
# Funcion para hacer la acumulacion de la varianza
var_cum = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
# Grafica para ver cuantos componentes necesitos para tener la mejor varianza, sin usar todos
px.area(
    x=range(1, var_cum.shape[0] + 1),
    y=(var_cum).round(2),
    labels={"x":"Components", "y":"Explained Variance"},
    title="Varianza Acumulada"
)

In [ ]:
# Grafico para ver que % aporta cada componente
fig_bar = px.bar(x=["PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "PC7", "PC8", "PC9"],
                 y=pca.explained_variance_ratio_*100,
                 title="Componentes principales",
                 labels={"x": "Components", "y":"Explained Variance"})
fig_bar.show()

In [ ]:
df_pca = pd.DataFrame(index=["PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "PC7", "PC8", "PC9"],
                      columns=df_country_scaled.columns, data=pca.components_)
df_pca

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
PC1,-0.419519,0.283897,0.150838,0.161482,0.398441,-0.193173,0.425839,-0.403729,0.392645
PC2,0.192884,0.613163,-0.243087,0.671821,0.022536,-0.008404,-0.222707,0.155233,-0.046022
PC3,-0.029544,0.144761,-0.596632,-0.299927,0.301548,0.642520,0.113919,0.019549,0.122977
PC4,0.370653,0.003091,0.461897,-0.071907,0.392159,0.150442,-0.203797,0.378304,0.531995
PC5,-0.168970,0.057616,0.518000,0.255376,-0.247150,0.714869,0.108220,-0.135262,-0.180167
PC6,0.200628,-0.059333,0.007276,-0.030032,0.160347,0.066285,-0.601127,-0.750689,0.016779
PC7,-0.079489,-0.707303,-0.249831,0.592190,0.095562,0.104633,0.018486,0.028826,0.242998
PC8,-0.682743,-0.014197,0.072497,-0.028946,0.352624,-0.011538,-0.504664,0.293353,-0.249696
PC9,0.327542,-0.123082,0.113088,0.099037,0.612982,-0.025236,0.294040,-0.026336,-0.625646


In [ ]:
fig_heatmap=px.imshow(df_pca,text_auto=True, aspect='auto', title="Componentes principales")
fig_heatmap.update_coloraxes(showscale=False) # Deshabilitar la barra de colores
fig.update_layout(title='Matriz de componentes principales', showlegend=False, height=1000, width=1000)
fig_heatmap.show()

---
<h5> Hasta ahora transforme los "Datos" de mi Dataset pero no los paises <h5>

---

In [ ]:
df_pca_country = pd.DataFrame(pca.transform(df_country_scaled), columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9'])

In [ ]:
# minuto 28:13
# Elimino los 5 ultimos componentes, para que queden los primeros 4 (87% de explicacion de la varianza)
df_pca_country.drop(['PC5', 'PC6', 'PC7', 'PC8', 'PC9'], axis=1, inplace=True)
df_pca_country

,PC1,PC2,PC3,PC4
0,-2.913025,0.095621,-0.718118,1.005255
1,0.429911,-0.588156,-0.333486,-1.161059
2,-0.285225,-0.455174,1.221505,-0.868115
3,-2.932423,1.695555,1.525044,0.839625
4,1.033576,0.136659,-0.225721,-0.847063
...,...,...,...,...
162,-0.820631,0.639570,-0.389923,-0.706595
163,-0.551036,-1.233886,3.101350,-0.115311
164,0.498524,1.390744,-0.238526,-1.074098
165,-1.887451,-0.109453,1.109752,0.056257


In [ ]:
# Algoritmo que me dice cuantos componentes necesito,
#segun el porcentaje de ganancia que quiero obtener en los resultados (Variacion acumulada)
var_cum = np.cumsum(pca.explained_variance_ratio_)*100
value = 90 #Aqui elijo el porcentaje de ganancia (variacion acumulada)
indx = np.where(var_cum >= value)[0][0]
print(f"Numero de componentes: {indx+1} con el {var_cum[indx].round(2)}%")

Numero de componentes: 5 con el 94.53%


---
<h4 align="center"> KMeans </h4>

---
Algoritmo para cluster (hay que indicarle la cantidad de cluster que necesitamos)-
Con PCA

In [ ]:
#======================================================================
# Funcion para graficar el metodo del codo y el coeficiente de silueta
#======================================================================
def elbow_silhouette(data, max_clusters):
  wcss = []
  sil = []
  n = np.arange(2,max_clusters)
  for i in n:
    kmeans = KMeans(n_clusters=i, random_state=42, n_init=10)
    y = kmeans.fit_predict(data)
    wcss.append(kmeans.inertia_)
    sil.append(silhouette_score(data,y))

  #Creamos las graficas
  fig = fig = make_subplots(rows=2, cols=1, shared_xaxes=True ,
                            x_title="Cantidad de clusters",
                            subplot_titles=("Elbow Method", "Silhouette Score"))
  fig.add_trace(go.Scatter(x=n,y=wcss,
                  mode='lines',
                  name="Inercia"),
                  row=1,
                  col=1)
  fig.add_trace(go.Scatter(x=n,y=sil,
                  mode='lines',
                  name="Score"),
                  row=2,
                  col=1)
  fig.update_layout(title='Elbow and Silhouette', showlegend=False)

  fig['layout']['yaxis']['title']='Inercia'
  fig['layout']['yaxis2']['title']='Silhouette Score'

  fig.show()


In [ ]:
# para indicarle a KMeans cuantos cluster necesitamos usamos la funcion de codo de shambu (Metodo de Elbow) o siluet
elbow_silhouette(df_pca_country, 10)

In [ ]:
# Creamos el modelo
km_pca = KMeans(n_clusters=4, n_init=10, random_state=42)
y_predict_pca = km_pca.fit_predict(df_pca_country)
print(silhouette_score(df_pca_country, y_predict_pca))

0.34930392685434886


In [ ]:
df_country_raw["KMeans_pca"] = y_predict_pca
df_country_raw
# Muestra en que cluster agrupo cada fila en la ultima columna

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp,KMeans_pca
0,Afghanistan,90.2,10.0,7.58,44.9,1610,9.44,56.2,5.82,553,1
1,Albania,16.6,28.0,6.55,48.6,9930,4.49,76.3,1.65,4090,2
2,Algeria,27.3,38.4,4.17,31.4,12900,16.10,76.5,2.89,4460,2
3,Angola,119.0,62.3,2.85,42.9,5900,22.40,60.1,6.16,3530,1
4,Antigua and Barbuda,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200,2
...,...,...,...,...,...,...,...,...,...,...,...
162,Vanuatu,29.2,46.6,5.25,52.7,2950,2.62,63.0,3.50,2970,2
163,Venezuela,17.1,28.5,4.91,17.6,16500,45.90,75.4,2.47,13500,2
164,Vietnam,23.3,72.0,6.84,80.2,4490,12.10,73.1,1.95,1310,2
165,Yemen,56.3,30.0,5.18,34.4,4480,23.60,67.5,4.67,1310,1


### Sin PCA

Comparando el coeficiente silhouette con PCA o Sin PCA, noto que el Con PCA me da 0,349 y SIN PCA 0,301.
Siempre hay que elegir el mas alto.

In [ ]:
elbow_silhouette(df_country_scaled, 10)

In [ ]:
# Creamos el modelo
km_pca = KMeans(n_clusters=4, n_init=10, random_state=42)
y_predict = km_pca.fit_predict(df_country_scaled)
print(silhouette_score(df_country_scaled, y_predict))

0.301375962376881


In [ ]:
df_country_raw["KMeans_sin_pca"] = y_predict
df_country_raw
# Muestra en que cluster agrupo cada fila en la ultima columna

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp,KMeans_pca,KMeans_sin_pca
0,Afghanistan,90.2,10.0,7.58,44.9,1610,9.44,56.2,5.82,553,1,1
1,Albania,16.6,28.0,6.55,48.6,9930,4.49,76.3,1.65,4090,2,0
2,Algeria,27.3,38.4,4.17,31.4,12900,16.10,76.5,2.89,4460,2,0
3,Angola,119.0,62.3,2.85,42.9,5900,22.40,60.1,6.16,3530,1,1
4,Antigua and Barbuda,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
162,Vanuatu,29.2,46.6,5.25,52.7,2950,2.62,63.0,3.50,2970,2,0
163,Venezuela,17.1,28.5,4.91,17.6,16500,45.90,75.4,2.47,13500,2,0
164,Vietnam,23.3,72.0,6.84,80.2,4490,12.10,73.1,1.95,1310,2,0
165,Yemen,56.3,30.0,5.18,34.4,4480,23.60,67.5,4.67,1310,1,1


In [ ]:
# Filtramos por grupos de KMeans_pca
filter = df_country_raw['KMeans_pca'] == 3
df_country_raw[filter]
# Vemos que el grupo 0 es de paises con mayores desarrollo humano
# grupo 1 paises con menor desarrollo humano
# Grupo 2 paises intermedio
# grupo 3 paises pequeños con grandes ingresos y desarrollo humano

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp,KMeans_pca,KMeans_sin_pca
91,Luxembourg,2.8,175.0,7.77,142.0,91700,3.620,81.3,1.63,105000,3,3
98,Malta,6.8,153.0,8.65,154.0,28300,3.830,80.3,1.36,21100,3,3
133,Singapore,2.8,200.0,3.96,174.0,72100,-0.046,82.7,1.15,46600,3,3


### Visualizacion

In [ ]:
#======================================================================
# Funcion para el mapa mundi
#======================================================================
def get_alpha_3(location):
    try:
        return pycountry.countries.get(name=location).alpha_3
    except:
        return None

In [ ]:
#@title Grafico del mundo
df_map = df_country_raw.copy()
df_map['Code'] = df_country_raw['country'].apply(lambda x: get_alpha_3(x))

fig = px.choropleth(df_map, locations='Code', color='KMeans_pca', hover_name='country', color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

In [ ]:
# @title Scatter plots
fig = make_subplots(rows=1, cols=3, subplot_titles=["child_mort vs life_expec", "exports vs imports", "gdpp vs child_mort"])

fig.add_trace(
    go.Scatter(x=df_country_raw["child_mort"], y=df_country_raw["life_expec"],
               mode="markers",
               name="Scatter 1",
               marker = dict(color=df_country_raw.KMeans_pca)),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df_country_raw["exports"], y=df_country_raw["imports"],
               mode="markers",
               name="Scatter 2",
               marker = dict(color=df_country_raw.KMeans_pca)),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=df_country_raw["gdpp"], y=df_country_raw["child_mort"],
               mode="markers",
               name="Scatter 3",
               marker = dict(color=df_country_raw.KMeans_pca)),
    row=1, col=3
)

fig.update_layout(title='Scatter plots', showlegend=False)
fig['layout']['xaxis']['title']='child_mort'
fig['layout']['yaxis']['title']='life_expec'
fig['layout']['xaxis2']['title']='exports'
fig['layout']['yaxis2']['title']='imports'
fig['layout']['xaxis3']['title']='gdpp'
fig['layout']['yaxis3']['title']='child_mort'

fig.show()

## Informacion mostrada a traves de HTML

In [ ]:
# @title iNFORMACION MOSTRADA a traves de Html
def grafica(df):
  f = go.Figure()
  for i, name in enumerate(df.columns):
    #print(i)

    f.add_trace(go.Box(y=df[f"{name}"], name=f"{name}", opacity=0.5))
  f.update_layout(height=900, width=1000, title_text="Charts", paper_bgcolor="#e4e9f7")

  # Renderiza la graficaen formato HTML
  graph_html = f.to_html(full_html=False, config={"displaylogo":False})
  return graph_html

def corr_matrix(df):
  # Calcula la matriz de correlacion
  correlation_matrix = df.corr().round(2)

  # Crear una visualizacion interativa usando Plotly
  fig = px.imshow(correlation_matrix,
                        x=correlation_matrix.columns,
                        y=correlation_matrix.columns,
                        color_continuous_scale='Viridis',
                        text_auto=True)
  graph_html = fig.to_html(full_html=False, config={"displaylogo":False})
  return graph_html

df_country_int = df_country_raw.select_dtypes(exclude="category") #Obtenemos solo las columnas con datos enteros para ver existencias de outliers

html_code = f"""
<div style="flex:1; ">
<p> Hagamos una EDA </p>
<h2> Datos numericos </h2>
{df_country_raw.describe().to_html()}
<p> De la tabla anterior podemos analizar rapidamente lo que es promedio, moda, media, etc. </p>
<h2>Datos Duplicados</h2>
{pd.DataFrame(df_country_raw[df_country_raw.duplicated()]).to_html()}
<p>Si no aparece nada es porque no poseemos datos duplicados</p>
</div>
<div style="flex:1; padding: 5px;">
  <h2> Nombre columnas</h2>
  {pd.DataFrame(df_country_raw.columns).to_html(index=False)}
</div>
<div style="flex: 1; padding: 5px">
  <h2>1 Nulos en cada columna</h2>
  {pd.DataFrame(df_country_raw.isnull().sum()).to_html()}
</div>
<div style="flex: 1; padding: 5px">
  <h2>Datos Duplicados</h2>
  {pd.DataFrame(df_country_raw[df_country_raw.duplicated()]).to_html()}
</div>
<div style="flex: 1;">
  <h2>Veamos si hay outliers y nos damos una idea de la distribucion de los datos</h2>
  {grafica(df_country_int)}
</div>
<div style="flex: 1;">
  <h2>Veamos la correlacion de datos</h2>
  {corr_matrix(df_country_int)}
</div>
"""
display(HTML(html_code))


NameError: ignored

Link: https://www.naftaliharris.com/blog/visualizing-k-means-clustering/

### Esta pagina nos muestra como elegir los centroides o dependiendo la distribucion de los datos que algoritmo nos conviene adoptar.
K-Means trabaja con distacias y DBScan por densidad

## VER DATA SET SINTETICOS EN MINUTO 40:22 min hasta

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html

In [ ]:
#======================================================================
# Funcion para graficar los clusters con matplotlib
#======================================================================
def plot_clusters_matplotlib(x, y, c, axe, title):
  #Seteamos valores de la grafica
  axe.set_title(title)
  axe.set_xlabel(f"{x.columns[0]}")
  axe.set_ylabel(f"{x.columns[1]}")
  #Obtenemos valores para graficar
  y_uniques = pd.Series(y).unique()
  x = np.array(x)
  y = np.array(y)
  for y_unique in y_uniques:
    axe.scatter(x[y == y_unique, 0], x[y == y_unique, 1], label=y_unique)
    axe.markers = [f"${y_unique}$"]
    axe.legend()
  axe.scatter(c[:, 0], c[:, 1], marker='x', s=200, linewidths=3, color='black', label='Centroids')
  axe.legend()

#======================================================================
# Funcion para graficar los clusters con plotly
#======================================================================
def plot_clusters_plotly(x, y, c, title):
  #Grafica
  fig = px.scatter(x, x=x.columns[0],
                   y=x.columns[1],
                   color=y,
                   title=title)
  #Agremos los cluster
  unique_labels = pd.Series(y).unique()
  for label in unique_labels:
      points = x[y == label]
      fig.add_trace(px.scatter(points, x=x.columns[0], y=x.columns[1], color=y).data[0])

  #Colocamos los centroides
  centroids_trace = px.scatter(c, x=x.columns[0], y=x.columns[1], color=unique_labels)
  centroids_trace.update_traces(marker=dict(symbol='x', size=12))
  fig.add_trace(centroids_trace.data[0])

  fig.show()

#======================================================================
# Funcion para graficar la matriz de correlacion con plotly
#======================================================================
def corr_matrix(df):
  # Calcular la matriz de correlación
  correlation_matrix = df.corr().round(2)

  # Crear una visualización interactiva usando Plotly
  fig = px.imshow(correlation_matrix,
                x=correlation_matrix.columns,
                y=correlation_matrix.columns,
                color_continuous_scale='Viridis',
                text_auto=True)
  fig.update_layout(title='Matriz de correlacion', showlegend=False, height=1000, width=1000)
  fig.show()